In [3]:
# today

# effective date -> start date

# end date

# Coupon Interval

# Bad day convention

# standard Cupons dates 

# A trade's first coupon payment date is determined by the trade date (T): it's the first coupon
# payment date after T+1 (calendar, unadjusted)

# The accrual period always stretches from (previous coupon payment date) through (this coupon
# payment date–1), inclusive; except a contract's last accrual period, which ends with (and
# includes) the unadjusted maturity date

# CDS Dates: 20th of Mar/Jun/Sep/Dec

In [70]:
import math
from dateutil import relativedelta
from datetime import datetime, timedelta

import rivapy
from rivapy.marketdata import DiscountCurve, DatedCurve, SurvivalCurve
from rivapy import enums
import pyvacon as _pyvacon
import scipy.optimize

In [5]:
def good_day_converter(date):
    # print(date) 
    if datetime.weekday(date) <6:
        start_date = date
    elif datetime.weekday(date) ==6:
        start_date = date + timedelta(days=2)
    elif datetime.weekday(date) ==7:
        start_date = date + timedelta(days=1)
    return(start_date)


In [6]:
# trade_date = 
# first_coupon_payment_date = earliest Adjusted CDS Date after T+1 calendar
# last_coupon_payment_date = latest Adjusted CDS Date on or before T+1 calendar
# next_coupon_payment_date = (this coupon payment date–1)
# maturity_date =  accrual end date is included unadjusted

In [7]:
def get_next_imm_date(date):
    imm_month_list=[3,6,9,12]
    next_imm_year = date.year
    if date.month in imm_month_list:
        if date.day < 20:
            next_imm_month=date.month
            # print(1)
        else:
            # print(2)
            next_date=date+relativedelta.relativedelta(months=3)
            next_imm_month = [min([m for m in imm_month_list if m >= next_date.month])][0]
            # print(next_imm_month)
            if next_imm_month < date.month:
                next_imm_year = next_imm_year+1
    else:
        # print(3)
        next_imm_month = [min([m for m in imm_month_list if m > date.month])][0]
    # print(dt.datetime(next_imm_year, next_imm_month, 20))
    next_imm_date = good_day_converter(datetime(next_imm_year, next_imm_month, 20))
    return (next_imm_date)

In [8]:
def get_previous_imm_date(date):
    previous_imm_date = date-relativedelta.relativedelta(months=3)
    return(good_day_converter(datetime(previous_imm_date.year, previous_imm_date.month, 20)))

In [9]:
# trade_date=datetime(2020,9,19)
# good_day_converter(datetime(2020, 12, 20))
# print(get_next_imm_date(trade_date))
# get_previous_imm_date(get_next_imm_date(trade_date))

In [10]:
def get_payment_dates(trade_date, maturity_date):   
    if maturity_date.day != 20:
        raise Exception('Maturity date needs to be CDS (IMM) date.')
    first_coupon_date = get_next_imm_date(trade_date)
    payment_dates = [first_coupon_date]
    i=0
    while payment_dates[-1]<maturity_date:
        next_date= get_next_imm_date(payment_dates[i])
        if next_date.month==maturity_date.month and next_date.year==maturity_date.year:
            payment_dates.append(maturity_date)
        else:
            payment_dates.append(get_next_imm_date(payment_dates[i]))
        i+=1
    return(payment_dates)

In [11]:
# trade_date = datetime(2008,3,20)
# maturity_date = datetime(2013,3,20)
# payment_dates=get_payment_dates(trade_date, maturity_date)

In [80]:
#Specs
trade_date = datetime(2007,3,20)
maturity_date = datetime(2013,6,20)
refdate = trade_date
premium_payment = 0.01
notional_amount=10000000

#yield curve
days_to_maturity = [0, 9, 15, 27, 39, 51, 63, 87, 123]
# rates = [-0.0065, 0.0003, 0.0059, 0.0086, 0.0101, 0.012, 0.016, 0.02]
days_to_maturity_ir=[1,2,3,6,9,12,24,36,48,60,72,84,96,108,120,132,144,180,240,300,360]
rates_ir=[0.004450, 0.009488, 0.012337, 0.017762, 0.019350, 0.020838, 0.016520, 0.020180, 0.023033, 0.025250, 0.026960, 0.028250, 
 0.029310, 0.030170, 0.030920, 0.031600, 0.032310, 0.033670, 0.034190, 0.034110, 0.034120]
dates = [refdate + relativedelta.relativedelta(months=d) for d in days_to_maturity]
dates_ir = [refdate + relativedelta.relativedelta(months=d) for d in days_to_maturity_ir]
dsc_fac = [math.exp(-rates_ir[i]*days_to_maturity_ir[i]/12) for i in range(len(days_to_maturity_ir))]
dsc_fac_ir = [1 for i in range(len(days_to_maturity_ir))]
dc = DiscountCurve('CDS_interest_rate', refdate, 
                                    dates_ir, 
                                    dsc_fac_ir,
                                    enums.InterpolationType.LINEAR)
# hazard_rates = [0, 0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.005]
# hazard_rates = [0, 0.023334991,0.067437529,0.149973856, 0.225205315, 0.293957702, 0.356445889, 0.465164897, 0.594880387]
hazard_rates = [0, 0.06722888, 0.10921745, 0.18781492, 0.25947741, 0.32498767, 0.38454683, 0.48851860, 0.61257003] 

#SurvivalCurve
sc = SurvivalCurve('Survival', refdate, dates, hazard_rates)

#RecoverCurve
# recoveries = [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]
recoveries = [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
recoveryCurve = DatedCurve('Recovery', refdate, dates, recoveries,
                                        enums.DayCounterType.Act365Fixed.name,
                                        enums.InterpolationType.LINEAR.name)

#Coupon Payment Dates
# payment_dates = [refdate + relativedelta.relativedelta(years=i) for i in range(10)]
payment_dates=get_payment_dates(trade_date, maturity_date)

#RiVaPy Spec
spec = rivapy.instruments.CDSSpecification(premium = premium_payment, protection_start=refdate, premium_pay_dates = payment_dates, notional = notional_amount)

#Pricind Data
cds_pricing_data = rivapy.pricing.CDSPricingData(spec=spec, val_date=refdate, discount_curve=dc, survival_curve=sc, recovery_curve=recoveryCurve)

#Price
pr = rivapy.pricing.price(cds_pricing_data)

In [81]:
# days_to_maturity_2 = [12, 18, 30, 42, 54, 66, 90, 126]
days_to_maturity_2 = [9, 15, 27, 39, 51, 63, 87, 123]
dates_2 = [refdate + relativedelta.relativedelta(months=d) for d in days_to_maturity_2]
hazard_rates_2 =    [0.0914442231726811,0.0932543950451530,0.0944780416613660,0.0949495158343314,0.0951701634832642,0.0953019935507213,0.0954976945248227,0.0956203078190794] 
# hazard_rates_2 =  [0.0974086731091612,0.0974086731091608,0.0974086731091606,0.0974086731091606,0.0974086731091608,0.0974086731091608,0.097408673109161,0.0974086731091608]
sc_2 = SurvivalCurve('Survival2', refdate, dates_2, hazard_rates_2)

In [82]:
payment_dates

[datetime.datetime(2007, 6, 20, 0, 0),
 datetime.datetime(2007, 9, 20, 0, 0),
 datetime.datetime(2007, 12, 20, 0, 0),
 datetime.datetime(2008, 3, 20, 0, 0),
 datetime.datetime(2008, 6, 20, 0, 0),
 datetime.datetime(2008, 9, 20, 0, 0),
 datetime.datetime(2008, 12, 20, 0, 0),
 datetime.datetime(2009, 3, 20, 0, 0),
 datetime.datetime(2009, 6, 20, 0, 0),
 datetime.datetime(2009, 9, 22, 0, 0),
 datetime.datetime(2009, 12, 22, 0, 0),
 datetime.datetime(2010, 3, 20, 0, 0),
 datetime.datetime(2010, 6, 22, 0, 0),
 datetime.datetime(2010, 9, 20, 0, 0),
 datetime.datetime(2010, 12, 20, 0, 0),
 datetime.datetime(2011, 3, 22, 0, 0),
 datetime.datetime(2011, 6, 20, 0, 0),
 datetime.datetime(2011, 9, 20, 0, 0),
 datetime.datetime(2011, 12, 20, 0, 0),
 datetime.datetime(2012, 3, 20, 0, 0),
 datetime.datetime(2012, 6, 20, 0, 0),
 datetime.datetime(2012, 9, 20, 0, 0),
 datetime.datetime(2012, 12, 20, 0, 0),
 datetime.datetime(2013, 3, 20, 0, 0),
 datetime.datetime(2013, 6, 20, 0, 0)]

In [83]:
cds_pricing_data = rivapy.pricing.CDSPricingData(spec=spec, val_date=refdate, discount_curve=dc, survival_curve=sc_2, recovery_curve=recoveryCurve)
pr = rivapy.pricing.price(cds_pricing_data)

In [84]:
print("Expected PV:", pr._price)
print("Expected PV protection leg:",pr.pv_protection)
print("Expected PV premium leg:",pr.premium_leg)
print("Expected PV accrued interest:", pr.accrued)
print("Expected PV premium leg + accrued:", pr.accrued+pr.premium_leg)

Expected PV: 2205035.506727943
Expected PV protection leg: 2678389.741304264
Expected PV premium leg: 467765.29634029494
Expected PV accrued interest: 5588.93823602545
Expected PV premium leg + accrued: 473354.2345763204


In [85]:
pr.par_spread

0.05658320018414072

In [86]:
def _pv_premium_leg(protection_start, premium_pay_dates, valuation_date, survival_curve, discount_curve ):
        premium_period_start = protection_start
        risk_adj_factor_premium=0  
        accrued = 0
        # print(premium_pay_dates)      
        #TODO include daycounter into CDSSpecification
        dc = _pyvacon.finance.definition.DayCounter(_pyvacon.finance.definition.DayCounter.Type.Act365Fixed)
        for premium_payment in premium_pay_dates:
            if premium_payment >= valuation_date:
                period_length = dc.yf(premium_period_start, premium_payment)
                survival_prob = survival_curve.value(valuation_date, premium_payment)
                # print(survival_prob)
                df = discount_curve.value(valuation_date, premium_payment)
                risk_adj_factor_premium += period_length*survival_prob*df
                # print(risk_adj_factor_premium*0.01*notional_amount)
                default_prob = survival_curve.value(valuation_date, premium_period_start)-survival_curve.value(valuation_date, premium_payment)
                accrued += 0.5*period_length*default_prob*df
                # print(accrued*0.5*0.01*notional_amount)
                premium_period_start = premium_payment
        return risk_adj_factor_premium, accrued


In [87]:
def _pv_protection_leg(val_date, protection_start, integration_step, expiry, survival_curve, recovery_curve, discount_curve):
        prev_date = max(val_date, protection_start+integration_step)
        current_date = min(prev_date + integration_step, expiry)
        pv_protection = 0.0
        
        while current_date <= expiry:
            default_prob = survival_curve.value(val_date, prev_date)-survival_curve.value(val_date, current_date)
            # print(survival_curve.value(val_date, prev_date))
            recovery = None
            if recovery is None and recovery_curve is not None:
                recovery = recovery_curve.value(val_date, current_date)
            # print(recovery)
            pv_protection += discount_curve.value(val_date, current_date)  * default_prob
            # print(default_prob)
            # print(pv_protection)
            prev_date = current_date
            current_date += integration_step
            
        if prev_date < expiry and current_date > expiry:
            default_prob = survival_curve.value(val_date, prev_date)-survival_curve.value(val_date, expiry)
            # print(survival_curve.value(val_date, expiry))
            recovery = recovery
            if recovery is None and recovery_curve is not None:
                recovery = recovery_curve.value(val_date, expiry) 
            pv_protection += discount_curve.value(val_date, expiry)  * default_prob
            # print(pv_protection)
            # print(default_prob)
            
        return pv_protection

In [88]:
pv_protection=_pv_protection_leg(refdate, refdate, relativedelta.relativedelta(months=3), maturity_date, sc_2, recoveryCurve, dc)*notional_amount* (1.0-0.4)
pv_premium=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[0]*premium_payment*notional_amount
pv_accrued=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[1]*premium_payment*notional_amount

In [89]:
pv_protection

2541677.6003533155

In [90]:
pv_premium

467765.296340295

In [91]:
pv = pv_protection-pv_premium-pv_accrued
pv

2068323.365776995

In [92]:
sc_2.value(datetime(2008,3,20),datetime(2008,3,20))

1.0

### Upfront Converter

In [93]:
# In: Upfront - PV of CDS
# Out: Fair Spread

# In: Fair spread
# Out: Upfront - PV of CDS

In [94]:
pv_protection=_pv_protection_leg(refdate, refdate, relativedelta.relativedelta(months=3), maturity_date, sc_2, recoveryCurve, dc)*notional_amount* (1.0-0.4)
pv_premium=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[0]*premium_payment*notional_amount
pv_accrued=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[1]*premium_payment*notional_amount
pv_CDS = pv_protection-pv_premium-pv_accrued
print(pv_CDS)

2068323.365776995


In [100]:
def calibration_error(x, upfront_charge, refdate, payment_dates, dc, sc_2, recoveryCurve):
    pv_protection=_pv_protection_leg(refdate, refdate, relativedelta.relativedelta(months=3), maturity_date, sc_2, recoveryCurve, dc)*notional_amount* (1.0-0.4)
    pv_premium=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[0]*x*notional_amount
    pv_accrued=_pv_premium_leg(refdate, payment_dates, refdate, sc_2, dc)[1]*x*notional_amount
    pv_CDS = pv_protection-pv_premium-pv_accrued
    return  upfront_charge - pv_CDS


def get_fair_spread(payment_dates, ref_date, premium_payment, upfront_charge, dc_new, sc_2, recoveryCurve):
    sol=scipy.optimize.root_scalar(calibration_error,args=( upfront_charge, ref_date, 
                         payment_dates, dc_new, sc_2, recoveryCurve),method='brentq',bracket=[-1,3],xtol=1e-8,rtol=1e-8)
    spread =  sol.root
        #print(numpy.exp(-sol.root*((payment_dates_iter[-1]-ref_date).days/365)))
    return spread

In [103]:
def calibration_error(x, upfront_charge, refdate, payment_dates, dc, standard_spread, recoveryCurve):
    pv_protection=_pv_protection_leg(refdate, refdate, relativedelta.relativedelta(months=3), maturity_date, x, recoveryCurve, dc)*notional_amount* (1.0-0.4)
    pv_premium=_pv_premium_leg(refdate, payment_dates, refdate, x, dc)[0]*standard_spread*notional_amount
    pv_accrued=_pv_premium_leg(refdate, payment_dates, refdate, x, dc)[1]*standard_spread*notional_amount
    pv_CDS = pv_protection-pv_premium-pv_accrued
    return  upfront_charge - pv_CDS


def get_fair_spread(payment_dates, ref_date, standard_spread, upfront_charge, dc_new, sc_2, recoveryCurve):
    sol=scipy.optimize.root_scalar(calibration_error,args=( upfront_charge, ref_date, 
                         payment_dates, dc_new, standard_spread, recoveryCurve),method='brentq',bracket=[-1,3],xtol=1e-8,rtol=1e-8)
    spread =  sol.root
        #print(numpy.exp(-sol.root*((payment_dates_iter[-1]-ref_date).days/365)))
    return spread

In [104]:
premium_payment = 0.01
notional_amount=10000000

upfront_charge = 240558.36

In [105]:
get_fair_spread(payment_dates, refdate, premium_payment, upfront_charge, dc, sc_2, recoveryCurve)

AttributeError: 'float' object has no attribute 'value'